## **06. Transformasi Data (Data Transformation)**

---
- Mengubah ke refesentasi vektor

- Membagi data ke datavalid, data testing, dan data training

## Import Library

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.preprocessing import StandardScaler
from gensim.models import Word2Vec
import numpy as np
from sklearn.preprocessing import StandardScaler
import os

Loading Data

In [9]:
data = pd.read_csv(
    '../05. Pra-pemrosesan Data (Data Preprocessing)/Dataset_Clean/Dataset_Clean.csv', 
    sep=","
)

In [19]:
data = pd.read_csv(
    '../06. Transformasi Data (Data Transformation)/Data_Split/train_data.csv', 
    sep=","
)

## A. Word Referesentasi (Word2vec)

### 1. Membentuk model Word2Vec

In [20]:
sentences = data['tweet'].apply(eval).tolist()

word2vec_model = Word2Vec(sentences, vector_size=8, window=5, min_count=1, workers=4)


### 2. Mengubah Teks menjadi Vektor

In [22]:
def tweet_to_vector(tweet, model):
    words = eval(tweet)  
    vector = np.mean([model.wv[word] for word in words if word in model.wv and word != '<pad>'], axis=0)
    if isinstance(vector, np.ndarray): 
        return vector
    else:
        return np.zeros(model.vector_size)  

data['tweet_vector'] = data['tweet'].apply(lambda x: tweet_to_vector(x, word2vec_model))


c:\Users\TEMP\miniconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\TEMP\miniconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [23]:
data

,Unnamed: 0,tweet,label,tweet_vector
0,31265,"['pemilu', 'momen', 'bersejarah', 'keseluruhan...",Joy,"[-0.20913523, -2.1562426, 0.886566, 2.5301132,..."
1,316,"['betul', 'juga', 'mikinya', 'kalau', 'waktu',...",Neutral,"[-0.9321435, -1.1031287, 1.7772691, 3.0841658,..."
2,4289,"['ngeri', 'sangat', 'jelas', 'positive', 'nong...",Fear,"[-1.2073029, -0.83843553, 1.5596131, 1.811603,..."
3,26820,"['yang', 'salah', 'cara', 'berpikir', 'yang', ...",Sad,"[0.10634192, -1.0936391, 0.24539249, 4.938823,..."
4,8377,"['acara', 'nya', 'seru', 'bange', 'waktu', 'ka...",Joy,"[-1.4973531, -0.8356001, 1.1920638, 2.6439862,..."
...,...,...,...,...
32734,6268,"['sopaa', 'cuz', 'akuu', 'suka', 'sekali', 'ju...",Love,"[-1.4871202, -0.9737764, 1.7366459, 2.8337927,..."
32735,19482,"['pak', 'anies', 'baswedan', 'cawapres', 'puny...",Love,"[-2.3848987, -2.018673, 2.7379007, 3.4556708, ..."
32736,39352,"['ngecek', 'link', 'ulang', 'benar', 'timnas',...",Joy,"[-2.643066, -1.0617319, 3.4989257, 2.2979307, ..."
32737,29727,"['biarpun', 'tim', 'mati', 'matian', 'bekerja'...",Anger,"[-0.5136224, -0.4523462, 0.5335757, 1.5281961,..."


### 3. Normalisasi fitur numerik

In [24]:
if 'numeric_feature' in data.columns:
    scaler = StandardScaler()
    data['normalized_numeric_feature'] = scaler.fit_transform(data[['numeric_feature']])

    data['features'] = data.apply(lambda row: np.concatenate((row['tweet_vector'], [row['normalized_numeric_feature']])), axis=1)
else:

    data['features'] = data['tweet_vector']


In [119]:
data

,Unnamed: 0,tweet,label,tweet_vector,features
0,0,"['pengabdian', 'terpuji', 'meskipun', 'terliha...",Neutral,"[1.1716549, -0.32019183, -0.52275145, -1.14394...","[1.1716549, -0.32019183, -0.52275145, -1.14394..."
1,1,"['baru', 'mengirim', 'foto', '<pad>', '<pad>',...",Neutral,"[2.0060577, -0.41663948, -2.1723745, -1.693770...","[2.0060577, -0.41663948, -2.1723745, -1.693770..."
2,2,"['apa', 'kabar', 'dunia', 'twitter', 'masihkah...",Neutral,"[1.59231, -0.48822427, -0.5230446, -1.1739731,...","[1.59231, -0.48822427, -0.5230446, -1.1739731,..."
3,3,"['yah', 'gimana', 'sih', 'yaudah', 'susul', 's...",Neutral,"[1.9332774, -1.4755193, -1.7834133, -1.4112929...","[1.9332774, -1.4755193, -1.7834133, -1.4112929..."
4,4,"['ka', 'dorg', 'suro', 'beli', 'medium', 'set'...",Neutral,"[0.6079169, -0.23392268, -1.0948995, -0.879356...","[0.6079169, -0.23392268, -1.0948995, -0.879356..."
...,...,...,...,...,...
28710,28710,"['ingin', 'sangat', 'nih', 'daerah', 'jakarta'...",Neutral,"[2.7875695, -1.5056438, -2.1028328, -2.1826487...","[2.7875695, -1.5056438, -2.1028328, -2.1826487..."
28711,28711,"['selamat', 'hari', 'raya', 'tri', 'suci', 'wa...",Joy,"[-0.43041828, 0.9478071, 0.070903234, -1.32057...","[-0.43041828, 0.9478071, 0.070903234, -1.32057..."
28712,28712,"['kandungan', 'serat', 'passion', 'fruit', 'ma...",Love,"[1.0247041, 0.12736644, -0.28464705, -1.146567...","[1.0247041, 0.12736644, -0.28464705, -1.146567..."
28713,28713,"['selamat', 'memperingati', 'hari', 'kebangkit...",Joy,"[0.3818012, 1.2018754, -0.13466965, -1.4089928...","[0.3818012, 1.2018754, -0.13466965, -1.4089928..."


In [25]:
data.to_csv("./Data_Transformed/data_transformed.csv", index=False)

## B. Membagi Dataset (Data Split)

Rasio

In [5]:
train_ratio = 0.7  # 70% untuk training
validation_ratio = 0.2  # 20% dari sisa 30% untuk validation, 10% untuk testing

Membagi Dataset

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# Membaca dataset
data = pd.read_csv(
    '../05. Pra-pemrosesan Data (Data Preprocessing)/Dataset_Clean/Dataset_Clean.csv', 
    sep=","
)

# Memisahkan fitur dan label
X = data.drop('label', axis=1)  # Ganti 'label' dengan nama kolom label Anda
y = data['label']  # Ganti 'label' dengan nama kolom label Anda

# Pembagian data menjadi training dan temporary set (untuk validation dan test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Pembagian temporary set menjadi validation dan test set
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# Menyimpan dataset yang telah dibagi ke direktori
output_dir = "./Data_Split"
os.makedirs(output_dir, exist_ok=True)

# Menyimpan training set
train_data = pd.concat([X_train, y_train], axis=1)
train_data.to_csv(os.path.join(output_dir, 'train_data.csv'), index=False)

# Menyimpan validation set
val_data = pd.concat([X_val, y_val], axis=1)
val_data.to_csv(os.path.join(output_dir, 'val_data.csv'), index=False)

# Menyimpan test set
test_data = pd.concat([X_test, y_test], axis=1)
test_data.to_csv(os.path.join(output_dir, 'test_data.csv'), index=False)

# Menampilkan ukuran setiap set
print(f'Training set size: {X_train.shape[0]}')
print(f'Validation set size: {X_val.shape[0]}')
print(f'Test set size: {X_test.shape[0]}')


Training set size: 32739
Validation set size: 7015
Test set size: 7016


In [35]:
import pandas as pd
import ast
from gensim.models import Word2Vec

# Load data from CSV file
data = pd.read_csv('../06. Transformasi Data (Data Transformation)/Data_Split/train_data.csv', sep=",")

# Convert the 'tweet' column from string representation of lists to actual lists
data['tweet'] = data['tweet'].apply(ast.literal_eval)

# Extract the sentences
sentences = data['tweet'].tolist()

# Train Word2Vec model with vector size of 8
model = Word2Vec(sentences, vector_size=8, window=5, min_count=1, workers=4)

# Get word vectors
word_vectors = model.wv

# Define the list of words to check
words = ['yang', 'salah', 'cara', 'berpikir', 'yang', 'berperang', 'melawan', 'penjajahan', 'israel', 'bukan', 'hamas', 'seluruh', 'rakyat', 'palestina']

# Check if each word is in the vocabulary and print its vector if present
for word in words:
    if word in word_vectors:
        print(f"Vector for '{word}':\n{word_vectors[word]}")
    else:
        print(f"Word '{word}' not found in the vocabulary.")


Vector for 'yang':
[-1.3604424  -1.6698306   2.387328    6.5467176  -0.40345243 -1.5080339
  5.1472936  -1.750812  ]
Vector for 'salah':
[-0.9371133 -2.4091237  0.6481962  7.5211425  0.8512062 -1.1003132
  4.120552  -3.2365217]
Vector for 'cara':
[-1.175257   -1.6943682   1.8103284   4.0345016   0.26900327 -1.9489814
  4.272454   -0.93799895]
Vector for 'berpikir':
[-0.45759773 -0.5999026   0.33058885  1.4528106   0.2907456  -0.34023815
  1.4839174   0.00278315]
Vector for 'yang':
[-1.3604424  -1.6698306   2.387328    6.5467176  -0.40345243 -1.5080339
  5.1472936  -1.750812  ]
Vector for 'berperang':
[ 0.24257535 -0.1995796  -0.09928071  0.6351126   0.5050839  -0.01318477
  0.3345605   0.04873794]
Vector for 'melawan':
[-0.09093627 -0.740151    0.19343987  2.5331702   0.9392481  -0.5056161
  1.8712153   0.13731764]
Vector for 'penjajahan':
[ 0.73953533 -0.31891137  0.1083969   1.5802063   0.9622953   0.0764325
  0.98203266  0.31435028]
Vector for 'israel':
[ 3.238653   1.3271939 -1.960